In [25]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime
# import plotly.io as pio
# pio.renderers.default = 'browser'

In [26]:
data = pd.read_csv("flux_tower_15days.csv", header = 0, low_memory = False)
skiprows = [0, 2, 3],
print(data.shape)
data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], format = '%Y-%m-%d %H:%M:%S')
print(data.dtypes)
data.set_index('TIMESTAMP', inplace = True, drop = False)
print(type(data.index))

(1296001, 21)
TIMESTAMP           datetime64[ns]
RECORD                       int64
Ux                         float64
Uy                         float64
Uz                         float64
co2                        float64
h2o                        float64
Ts                         float64
press                      float64
diag_csat                    int64
t_hmp                      float64
rh_hmp                     float64
e_hmp                      float64
fw                         float64
short_up                   float64
short_dn                   float64
long_up                    float64
long_dn                    float64
cnr4_T_K                   float64
long_up_corr_Avg           float64
long_dn_corr_Avg           float64
dtype: object
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [27]:
day_mask = (data.index.time >= datetime.strptime('08:30', '%H:%M').time()) & \
           (data.index.time <= datetime.strptime('17:30', '%H:%M').time())

night_mask = (data.index.time >= datetime.strptime('21:30', '%H:%M').time()) | \
           (data.index.time <= datetime.strptime('05:30', '%H:%M').time())

day_data = data[day_mask]
night_data = data[night_mask]

In [28]:
def parse_datetime(timestamp):
    try:
        return pd.to_datetime(timestamp, format = '%Y-%m-%d %H:%M:%S')
    except ValueError:
        return pd.to_datetime(timestamp, format = '%Y-%m-%d %H:%M:%S.%f')
data['TIMESTAMP'] = data['TIMESTAMP'].apply(parse_datetime)
print(type(data.index))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [29]:
def plotting(data, title_suffix):
    data['10min'] = data.index.floor('10T')

    grouped_data_u = data.groupby('10min').mean().rename(columns = {'Ux': 'Ux_avg'})
    aligned_avg_data_u = grouped_data_u.reindex(data.index, method = 'ffill')

    grouped_data_v = data.groupby('10min').mean().rename(columns = {'Uy': 'Uy_avg'})
    aligned_avg_data_v = grouped_data_v.reindex(data.index, method = 'ffill')

    grouped_data_w = data.groupby('10min').mean().rename(columns = {'Uz': 'Uz_avg'})
    aligned_avg_data_w = grouped_data_w.reindex(data.index, method = 'ffill')

    # Calculate Mean Kinetic Energy (MKE)
    MKE = 0.5 * (aligned_avg_data_u['Ux_avg']**2 + aligned_avg_data_v['Uy_avg']**2 + aligned_avg_data_w['Uz_avg']**2)

    # Calculate fluctuations
    fluctuation_u = data['Ux'] - aligned_avg_data_u['Ux_avg']
    fluctuation_v = data['Uy'] - aligned_avg_data_v['Uy_avg']
    fluctuation_w = data['Uz'] - aligned_avg_data_w['Uz_avg']

    # Calculate Turbulent Kinetic Energy (TKE)
    TKE = 0.5 * (fluctuation_u**2 + fluctuation_v**2 + fluctuation_w**2)

    # Calculate Turbulence Intensity (TKE/MKE)
    turbulence_intensity = TKE / MKE

    # Plotting
    fig_U_mean = px.line(x = data['TIMESTAMP'][:86400], y = aligned_avg_data_u['Ux_avg'][:86400], title = f'U Mean {title_suffix}')
    fig_U_mean.show()

    fig_U = px.line(x = data['TIMESTAMP'][:86400], y = fluctuation_u[:86400], title = f'U Fluctuation {title_suffix}')
    fig_U.show()

    fig_V_mean = px.line(x = data['TIMESTAMP'][:86400], y = aligned_avg_data_v['Uy_avg'][:86400], title = f'V Mean {title_suffix}')
    fig_V_mean.show()

    fig_V = px.line(x = data['TIMESTAMP'][:86400], y = fluctuation_v[:86400], title = f'V Fluctuation {title_suffix}')
    fig_V.show()

    fig_W_mean = px.line(x = data['TIMESTAMP'][:86400], y = aligned_avg_data_w['Uz_avg'][:86400], title = f'W Mean {title_suffix}')
    fig_W_mean.show()

    fig_W = px.line(x = data['TIMESTAMP'][:86400], y = fluctuation_w[:86400], title = f'W Fluctuation {title_suffix}')
    fig_W.show()

    # Plot MKE and TKE
    fig_MKE = px.line(x = data['TIMESTAMP'][:86400], y = MKE[:86400], title = f'Mean Kinetic Energy (MKE) {title_suffix}')
    fig_MKE.show()

    fig_TKE = px.line(x = data['TIMESTAMP'][:86400], y = TKE[:86400], title = f'Turbulent Kinetic Energy (TKE) {title_suffix}')
    fig_TKE.show()

    # Plot Turbulence Intensity (TKE/MKE)
    fig_turbulence_intensity = px.line(x = data['TIMESTAMP'][:86400], y = turbulence_intensity[:86400], title = f'Turbulence Intensity {title_suffix}')
    fig_turbulence_intensity.show()



In [30]:
# Calculate and plot for day data
plotting(day_data, 'Day')

# Calculate and plot for night data
plotting(night_data, 'Night')

/var/folders/hn/ff472bw55p9dv_cxds0qb_wh0000gn/T/ipykernel_38262/143488777.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



KeyboardInterrupt: 

In [ ]:
# data['10min'] = data.index.floor('10T')

# grouped_data_u = data.groupby('10min').mean().rename(columns = {'Ux': 'Ux_avg'})
# aligned_avg_data_u = grouped_data_u.reindex(data.index, method = 'ffill')

# fig_U_mean = px.line(x=data['TIMESTAMP'][:86400],y=aligned_avg_data_u['Ux_avg'][:86400])
# fig_U_mean.show()

# fluctuation_u = data['Ux'] - aligned_avg_data_u['Ux_avg']
# fig_U = px.line(x=data['TIMESTAMP'][:86400],y=fluctuation_u[:86400])
# fig_U.show()

# grouped_data_v = data.groupby('10min').mean().rename(columns = {'Uy': 'Uy_avg'})
# aligned_avg_data_v = grouped_data_v.reindex(data.index, method = 'ffill')

# fig_V_mean = px.line(x=data['TIMESTAMP'][:86400],y=aligned_avg_data_v['Uy_avg'][:86400])
# fig_V_mean.show()

# fluctuation_v = data['Uy'] - aligned_avg_data_v['Uy_avg']
# fig_V = px.line(x=data['TIMESTAMP'][:86400],y=fluctuation_v[:86400])
# fig_V.show()

# grouped_data_w = data.groupby('10min').mean().rename(columns = {'Uz': 'Uz_avg'})
# aligned_avg_data_w = grouped_data_w.reindex(data.index, method = 'ffill')
# fig_W_mean = px.line(x=data['TIMESTAMP'][:86400],y=aligned_avg_data_w['Uz_avg'][:86400])
# fig_W_mean.show()

# fluctuation_w = data['Uz'] - aligned_avg_data_w['Uz_avg']
# fig_W = px.line(x=data['TIMESTAMP'][:86400],y=fluctuation_w[:86400])
# fig_W.show()